# ddi - Mini-Challenge zu LE3, NoSQL
Roman Studer, Simon Luder

## Use Case

## Datenmodell

## Setup MongoDB

In [ ]:
#!pip install pymongo
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pymongo import MongoClient
import pandas as pd
import pymongo
import os

### Prepare data

In [ ]:
def csv_to_dict(path, file):
    '''converts a csv file to a dictionary'''
    data = pd.read_csv(path + file)
    pd.to_datetime(data.current_time, format="%Y-%m-%d %H:%M:%S")
    dictionary = dict()
    dictionary["_id"] = file.replace('.csv', '')
    dictionary["box_name"] = data["box_name"][0]
    dictionary["sensor_name"] = data["sensor_name"][0]
    dictionary["box_id"] = data["box_id"][0]
    dictionary["sensor_id"] = data["sensor_id"][0]
    dictionary["lat"] = data["lat"][0]
    dictionary["lon"] = data["lon"][0]
    dictionary["unit"] = data["unit"][0]
    dictionary["measurements"] = dict(zip(data["current_time"], data["value"]))
    return dictionary


### Create db

In [ ]:
mongo_client = pymongo.MongoClient('localhost', 27017)
mongo_db = mongo_client["ddi_mc2"]

### Populate db

In [ ]:
eth = mongo_db['ETH']
path = './data/'
for file in os.listdir("./data"):
#     print(file.split("_")[0])
    if not mongo_db[file.split("_")[0]].count_documents({"_id":file.replace('.csv', '')}) > 0:
        dictionary = csv_to_dict(path, file)
        mongo_db[file.split("_")[0]].insert_one(dictionary)
        print("populate:", file)

In [ ]:
mongo_db["IfU SenseBox2021 2B"].find_one({"unit":"%"})

## Setup InfluxDB 

In [ ]:
#!pip install influxdb
from influxdb import InfluxDBClient
influx_client = InfluxDBClient(host='localhost', port=8086)